In [1]:
%load_ext autoreload
%autoreload 2

# Load samples

In [2]:
from pydantic import BaseModel, Field
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from AutoLLM._utils.general import load_df
from AutoLLM.prompts.classifier import ClassifierPrompt
from AutoLLM.prompts.sample import SampleConfig, SampleItem, ConfigItem
from AutoLLM.prompts.few_shot_samples import FewShotSamples
from AutoLLM.prompts.input import InputSample

TypeError: 'type' object is not subscriptable

In [ ]:
df = load_df("./_data/Ecommerce/ecommerceDataset.csv")
df

,label,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50424,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50425,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50426,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50427,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [7]:
sc = SampleConfig(
    configs=[
        ConfigItem(name="Item Description", description="Description of the item"),
        ConfigItem(name="Type of item", description="Answer as one of the following prompts only: {Clothing & Accessories, Electronics, Books, Household}"),
    ]
)

sample_item_list = []
for i, r in tqdm(df.iterrows(), total=len(df)):
    # print(r["text"])
    sample = SampleItem(
        sample_config=sc, 
        **{
            "Item Description": r['text'],
            "Type of item": r['label'],
        }
    )

    sample_item_list.append(sample)
df['sample_item'] = sample_item_list
df['input_text'] = df['sample_item'].apply(lambda x: InputSample(input_sample=x))
df

  0%|          | 0/50429 [00:00<?, ?it/s]

,label,text,sample_item,input_text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...,SampleItem(data={'Item Description': 'Paper Pl...,input_sample=SampleItem(data={'Item Descriptio...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...","SampleItem(data={'Item Description': ""SAF 'Flo...",input_sample=SampleItem(data={'Item Descriptio...
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...,"SampleItem(data={'Item Description': ""SAF 'UV ...",input_sample=SampleItem(data={'Item Descriptio...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1...",SampleItem(data={'Item Description': 'SAF Flow...,input_sample=SampleItem(data={'Item Descriptio...
4,Household,Incredible Gifts India Wooden Happy Birthday U...,SampleItem(data={'Item Description': 'Incredib...,input_sample=SampleItem(data={'Item Descriptio...
...,...,...,...,...
50424,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...,SampleItem(data={'Item Description': 'Strontiu...,input_sample=SampleItem(data={'Item Descriptio...
50425,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...,SampleItem(data={'Item Description': 'CrossBea...,input_sample=SampleItem(data={'Item Descriptio...
50426,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...,SampleItem(data={'Item Description': 'Karbonn ...,input_sample=SampleItem(data={'Item Descriptio...
50427,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou...",SampleItem(data={'Item Description': 'Samsung ...,input_sample=SampleItem(data={'Item Descriptio...


Item Description: Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some

In [11]:
import requests

url = "http://127.0.0.1:8000/v1/completions"
json_data = {
    "prompt": "what are you thinking?",
    "temperature": 1e-10,
    "top_p": 0.9,
    "top_k": 40,
    "max_new_tokens": 100,
    "do_sample": True,
    "verbose": True,
}

response = requests.post(url, json=json_data)
response.json()

{'id': 'cmpl-unique-id',
 'object': 'text_completion',
 'created': 123456789,
 'content': " are you thinking about the upcoming election? are you thinking about the latest news? are you thinking about the latest trends? are you thinking about the latest memes? are you thinking about the latest memes about the latest memes? \n\nI'm thinking about the fact that I'm thinking about the fact that I'm thinking about the fact that I think I'm thinking about the fact that I'm thinking about the fact that I'm thinking about the fact that I'm thinking about the fact that I'm thinking about",
 'throughput': 4.42968627946432}